### Save models architecture in JSON format

could be used to load without models code available directly.

models saved in DIR_MODELS_ARCH

In [1]:
!pip install efficientnet >> /dev/null

In [1]:
import tensorflow as tf
import efficientnet.tfkeras as efn
import glob

In [2]:
print(tf.__version__)

2.1.0


In [3]:
DIR_MODELS = './tpu-models'
DIR_MODELS_ARCH = './models-arch'

IMG_SIZES = 512

IMAGE_SIZE = [IMG_SIZES, IMG_SIZES]

# tune it, dependes on Image, size, TPU or GPU
BATCH_SIZE = 26

# for K-fold CV
FOLDS = 5

# WHICH EFFICIENTNET TO USE (B?, B0 from B7)
EFF_NETS = 4

In [4]:
list_models = sorted(glob.glob(DIR_MODELS + '/*.h5'))

print(list_models)

['./tpu-models/fold-1.h5', './tpu-models/fold-2.h5', './tpu-models/fold-3.h5', './tpu-models/fold-4.h5', './tpu-models/fold-5.h5']


In [5]:
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

# as default it used B0

def build_model(dim = IMG_SIZES, ef = 0):
    inp = tf.keras.layers.Input(shape=(*IMAGE_SIZE, 3))
    
    base = EFNS[ef](input_shape=(*IMAGE_SIZE, 3), weights='imagenet', include_top = False)
    
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs = inp,outputs = x)
    
    opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
    
    # removed label smoothing
    fn_loss = tf.keras.losses.BinaryCrossentropy() 
    
    # loss = [focal_loss]
    model.compile(optimizer = opt, loss = [fn_loss], metrics=['AUC', 'accuracy'])
    
    return model

In [6]:
try_model = build_model(dim=IMG_SIZES, ef = EFF_NETS)

try_model.summary()

71892992/71892840 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
efficientnet-b4 (Model)      (None, 16, 16, 1792)      17673816  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               918016    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 18,592,345
Trainable params: 18,467,145
Non-trainable params: 125,200
_________________________________________________________________


In [7]:
for i, f_name in enumerate(list_models):
    print('Loading model from:', f_name)
    
    model = build_model(dim=IMG_SIZES, ef=EFF_NETS)
    
    model.load_weights(f_name)
    
    # saving model architecture
    print('Saving model arch. for:', f_name)
    
    model_json = model.to_json()
    arch_name = DIR_MODELS_ARCH + "/fold-" + str(i+1) + ".json"
    
    with open(arch_name, "w") as json_file:
        json_file.write(model_json)

Loading model from: ./tpu-models/fold-1.h5
Saving model arch. for: ./tpu-models/fold-1.h5
Loading model from: ./tpu-models/fold-2.h5
Saving model arch. for: ./tpu-models/fold-2.h5
Loading model from: ./tpu-models/fold-3.h5
Saving model arch. for: ./tpu-models/fold-3.h5
Loading model from: ./tpu-models/fold-4.h5
Saving model arch. for: ./tpu-models/fold-4.h5
Loading model from: ./tpu-models/fold-5.h5
Saving model arch. for: ./tpu-models/fold-5.h5


In [8]:
!ls -l $DIR_MODELS_ARCH

total 1140
-rw-r--r-- 1 root root 225295 Jan  6 22:55 fold-1.json
-rw-r--r-- 1 root root 225295 Jan  6 22:56 fold-2.json
-rw-r--r-- 1 root root 225295 Jan  6 22:56 fold-3.json
-rw-r--r-- 1 root root 225299 Jan  6 22:56 fold-4.json
-rw-r--r-- 1 root root 225309 Jan  6 22:56 fold-5.json
